In [91]:
import os
import cv2
import random
import numpy as np
from natsort import natsorted
import math

class Image_generator:
  def __init__(self, img_size=(400, 600), nof_imgs=3, nof_cells=50):
    self.__img_size = img_size
    self.__nof_cells = nof_cells
    self.__nof_imgs = nof_imgs

  def generate(self, ptrns_dir="C:/Users/looki/Space/Institute/МФТИ/Python/Dev/"
               "Semester_10/hw_2/Patterns"):
    ptrns = {}
    self.__generated = []
    print(ptrns_dir)
    for root, dirs, files in os.walk(ptrns_dir):
      if len(files) != 0:
        lable = root.replace('\\', '/')
        ptrns[lable] = natsorted(files)

    for _ in range(self.__nof_imgs):
      res_img = self.__generate_img(ptrns)
      self.__generated.append(res_img)

  def __generate_img(self, ptrns):
    cells_coords = self.__gen_cells_coords()

    os.chdir(list(ptrns.keys())[0])
    back_img = cv2.imread(random.choice(ptrns[list(ptrns.keys())[0]]))

    os.chdir(list(ptrns.keys())[1])
    cell_img = cv2.imread(random.choice(ptrns[list(ptrns.keys())[1]]),
                          cv2.IMREAD_UNCHANGED)

    for idx in range(self.__nof_cells):
      back_img = self.__overlay(back_img, cell_img, 
                                {"h": cells_coords["h"][idx], 
                                 "w": cells_coords["w"][idx]})
      
    return back_img

  def __gen_cells_coords(self):
    coords = {"h": [], "w": []}

    for _ in range(self.__nof_cells):
      coords["h"].append(random.randint(0, self.__img_size[0] -
                                        int(self.__img_size[0] * 0.01)))
      coords["w"].append(random.randint(0, self.__img_size[1] -
                                        int(self.__img_size[1] * 0.01)))
      
    return coords
  
  def __overlay(self, back_img, cell_img, coord):
    scale_coef = round(random.uniform(0.20, 0.05), 2)

    back_img = cv2.resize(back_img, dsize=(self.__img_size[1], 
                                           self.__img_size[0]), 
                          interpolation=cv2.INTER_CUBIC)
    
    size_min = min(self.__img_size)
    cell_h = int(size_min * scale_coef)
    cell_w = int(size_min * scale_coef)
    cell_img = cv2.resize(cell_img, dsize=(cell_w, cell_h),
                          interpolation=cv2.INTER_CUBIC)
    
    cell_img, cell_h, cell_w = self.__rotate_cell(cell_img, cell_h, cell_w)

    down_shift = 0
    if coord["h"] + cell_h > back_img.shape[0]:
      down_shift = coord["h"] + cell_h - back_img.shape[0]
    right_shift = 0
    if coord["w"] + cell_w > back_img.shape[1]:
      right_shift = coord["w"] + cell_w - back_img.shape[1]
    cell_img = cell_img[:cell_h - down_shift, :cell_w - right_shift]
    
    cell_img_rgb = cell_img[:, :, :3]
    cell_img_alpha = cell_img[:, :, 3] / 255.0
    region = back_img[coord["h"]:coord["h"] + cell_h - down_shift, 
                      coord["w"]:coord["w"] + cell_w - right_shift]
    
    for chnl in range(3):
      region[:, :, chnl] = (1.0 - cell_img_alpha) * region[:, :, chnl] + \
        cell_img_alpha * cell_img_rgb[:, :, chnl]
    
    back_img[coord["h"]:coord["h"] + cell_h - down_shift, 
             coord["w"]:coord["w"] + cell_w - right_shift] = region
      
    return back_img
  
  def __rotate_cell(self, cell_img, cell_h, cell_w):
    img_center = (cell_w // 2, cell_h // 2)
    rotate_angle = random.randint(0, 90)
    rotation_matrix = cv2.getRotationMatrix2D(img_center, rotate_angle, 1.0)

    cell_w = int((cell_h * np.abs(rotation_matrix[0, 1])) + \
                 (cell_w * np.abs(rotation_matrix[0, 0])))
    cell_h = int((cell_h * np.abs(rotation_matrix[0, 0])) + \
                 (cell_w * np.abs(rotation_matrix[0, 1])))
    
    rotation_matrix[0, 2] += (cell_w / 2) - img_center[0]
    rotation_matrix[1, 2] += (cell_h / 2) - img_center[1]

    cell_img = cv2.warpAffine(cell_img, rotation_matrix, (cell_w, cell_h))

    return cell_img, cell_h, cell_w


  def __getitem__(self,key):
    return self.__generated[key]
  
  def save_to_dir(self, save_dir):
    os.chdir(save_dir)
    for idx in range(len(self.__generated)):
      cv2.imwrite(f"res_{idx}.png", self.__generated[idx])


In [92]:
gen_imgs = Image_generator(nof_imgs=10, nof_cells=100)
gen_imgs.generate()

C:/Users/looki/Space/Institute/МФТИ/Python/Dev/Semester_10/hw_2/Patterns


In [93]:
gen_imgs.save_to_dir("C:\\Users\\looki\\Space\\Institute\\МФТИ\\Python\\Dev\\Semester_10\\hw_2\\res".replace('\\', '/'))